### Retrieval-Augmented Generation (RAG) using EigenDB

Dataset: https://huggingface.co/ngxson/demo_simple_rag_py/blob/main/cat-facts.txt

In [1]:
from eigen_py.index import Index
from eigen_py.data_types import Document

from ollama import chat
from dotenv import load_dotenv
import os
import re

load_dotenv()  

True

In [2]:
LLM_NAME = "tinyllama:latest" # using Ollama for LLM inference

In [3]:
index = Index(
    url="http://localhost:8080",
    api_key="7fd845ada2798ba93e25fa3c93b267f4",
    model_name="text-embedding-3-small",
    model_provider="openai",
    model_provider_api_key=os.getenv("OPENAI_TOKEN")
)
index

Authentication successful: Authenticated.


EigenDB_Index(url=http://localhost:8080)

In [4]:
def query_llm(query: str) -> str:
    res = chat(
        model=LLM_NAME, 
        messages=[
            {
                'role': 'system', 
                'content': 'You are a helpful assistant that answers questions using any provided context and your own knowledge to help your answer. If you are unsure, say "I do not know". Don\'t make up answers.'
            }, 
            {
                'role': 'user', 
                'content': query
            }
        ]
    )
    response = res['message']['content']
    return response


In [5]:
with open('datasets/cat_facts/cat-facts.txt', 'r') as f:
    facts = f.readlines()
    facts = [Document(id=i, data=re.sub(r'\n', '', facts[i]), metadata={}) for i in range(len(facts))]
len(facts)

150

In [6]:
index.upsert_docs(facts)

In [9]:
def query_rag_pipeline(query: str, k: int = 5) -> str:
    results = index.search_docs(string=query, k=k)
    context = "\n".join([facts[int(embedding_id)].data for embedding_id in results.keys()])
    full_query = f"CONTEXT:\n{context}\n\nQUESTION:\n{query}"
    return query_llm(full_query), context

In [10]:
answer, context = query_rag_pipeline("Were cats very important in ancient Egypt?")
print(f"ADDED CONTEXT:\n{context}\n\n")
print(f"ANSWER:\n{answer}")

ADDED CONTEXT:
While it is commonly thought that the ancient Egyptians were the first to domesticate cats, the oldest known pet cat was recently found in a 9,500-year-old grave on the Mediterranean island of Cyprus. This grave predates early Egyptian art depicting cats by 4,000 years or more.
In ancient Egypt, killing a cat was a crime punishable by death.
In ancient Egypt, mummies were made of cats, and embalmed mice were placed with them in their tombs. In one ancient city, over 300,000 cat mummies were found.
When a family cat died in ancient Egypt, family members would mourn by shaving off their eyebrows. They also held elaborate funerals during which they drank wine and beat their breasts. The cat was embalmed with a sculpted wooden mask and the tiny mummy was placed in the family tomb or in a pet cemetery with tiny mummies of mice.
Smuggling a cat out of ancient Egypt was punishable by death. Phoenician traders eventually succeeded in smuggling felines, which they sold to rich pe